In [1]:
# Parameters
alpha = 0.6
ratio = 0.1


In [2]:
import glob
import os
import pandas as pd
import numpy as np


In [3]:
import os
import pandas as pd

# Ruta base a las carpetas
base_path = os.path.join('..', '..', 'data_backup', 'archivos_parquet')

# Lista de carpetas a procesar
folders = [
    'Copa Vasca Cadete',
    'D. Honor Juvenil - G2',
    'División de Honor - Araba',
    'Liga Nacional Juvenil - Grupo 4',
    'Liga_Vasca_Cadete',
    'Segunda RFEF - G2',
    'Tercera RFEF - G4'
]

def get_file_prefix(filename):
    # Para archivos con número al final
    parts = filename.replace('.parquet', '').split('_')
    if parts[-1].isdigit():
        return '_'.join(parts[:-1])
    # Para archivos sin número
    return filename.replace('.parquet', '')

print(f"Los archivos combinados se guardarán en: {base_path}")

# Diccionario para almacenar todos los DataFrames por tipo de archivo
all_files_dict = {}
# Diccionario para archivos que aparecen en todas las carpetas
common_files_dict = {}

# Primero, encontrar archivos que aparecen en todas las carpetas
all_folder_files = {}
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    try:
        files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]
        all_folder_files[folder] = set(files)
    except Exception as e:
        print(f"Error al acceder a la carpeta {folder}: {str(e)}")
        continue

# Encontrar archivos comunes a todas las carpetas
common_files = set.intersection(*all_folder_files.values())
print(f"\nArchivos comunes encontrados: {common_files}")

# Procesar archivos normalmente y recolectar los comunes
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    print(f"\nProcesando carpeta: {folder}")
    
    try:
        files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]
        print(f"Encontrados {len(files)} archivos parquet en {folder}")
    except Exception as e:
        print(f"Error al acceder a la carpeta {folder}: {str(e)}")
        continue
    
    # Agrupar archivos por prefijo
    for file in files:
        prefix = get_file_prefix(file)
        if prefix:
            # Para archivos con números
            if prefix not in all_files_dict:
                all_files_dict[prefix] = []
            file_path = os.path.join(folder_path, file)
            
            try:
                df = pd.read_parquet(file_path)
                # Si es un archivo común, guardarlo en common_files_dict
                if file in common_files:
                    if file not in common_files_dict:
                        common_files_dict[file] = []
                    common_files_dict[file].append(df)
                else:
                    all_files_dict[prefix].append(df)
                print(f"Leído: {file} - {len(df)} filas")
            except Exception as e:
                print(f"Error al leer {file}: {str(e)}")

# Procesar archivos normales
for prefix, dfs in all_files_dict.items():
    if len(dfs) > 0:
        print(f"\nCombinando archivos con prefijo: {prefix}")
        
        combined_df = pd.concat(dfs, ignore_index=True)
        
        original_name = next(name for name in os.listdir(folder_path) 
                           if name.startswith(prefix) and name.endswith('.parquet'))
        if original_name.replace('.parquet', '').split('_')[-1].isdigit():
            output_name = f"{prefix}_all.parquet"
        else:
            output_name = f"{prefix}.parquet"
            
        output_path = os.path.join(base_path, output_name)
        combined_df.to_parquet(output_path, index=False)
        print(f"Guardado: {output_path}")
        print(f"Total filas en archivo combinado: {len(combined_df)}")
        print(f"Columnas: {combined_df.columns.tolist()}")

# Procesar archivos comunes
for file, dfs in common_files_dict.items():
    if len(dfs) > 0:
        print(f"\nCombinando archivo común: {file}")
        
        combined_df = pd.concat(dfs, ignore_index=True)
        
        # Crear nombre con _all
        name_without_ext = file.replace('.parquet', '')
        output_name = f"{name_without_ext}_all.parquet"
        
        output_path = os.path.join(base_path, output_name)
        combined_df.to_parquet(output_path, index=False)
        print(f"Guardado: {output_path}")
        print(f"Total filas en archivo combinado: {len(combined_df)}")
        print(f"Columnas: {combined_df.columns.tolist()}")

print("\nProceso completado")

Los archivos combinados se guardarán en: ../../data_backup/archivos_parquet

Archivos comunes encontrados: {'seasons_teams.parquet', 'leagues.parquet'}

Procesando carpeta: Copa Vasca Cadete
Encontrados 12 archivos parquet en Copa Vasca Cadete
Leído: players_league_1992.parquet - 43 filas
Leído: matches_league_1992.parquet - 5 filas
Leído: sequence_data_league_1992.parquet - 588 filas
Leído: teams_league_1992.parquet - 8 filas
Leído: names_players_league_1992.parquet - 37 filas
Leído: events_league_1992.parquet - 1811 filas
Leído: team_stats_league_1992.parquet - 2 filas
Leído: leagues.parquet - 6 filas
Leído: lineups_league_1992.parquet - 37 filas
Leído: formations_league_1992.parquet - 98 filas
Leído: seasons_teams.parquet - 15 filas
Leído: player_stats_extended_league_1992.parquet - 29 filas

Procesando carpeta: D. Honor Juvenil - G2
Encontrados 12 archivos parquet en D. Honor Juvenil - G2
Leído: teams_league_698.parquet - 69 filas
Leído: player_stats_extended_league_698.parquet - 4

Leído: events_league_1179.parquet - 234231 filas
Leído: team_stats_league_1179.parquet - 250 filas
Leído: players_league_1179.parquet - 2208 filas
Leído: sequence_data_league_1179.parquet - 68810 filas
Leído: matches_league_1179.parquet - 246 filas
Leído: leagues.parquet - 6 filas
Leído: player_stats_extended_league_1179.parquet - 3833 filas
Leído: seasons_teams.parquet - 15 filas
Leído: lineups_league_1179.parquet - 4479 filas
Leído: formations_league_1179.parquet - 11453 filas

Procesando carpeta: Tercera RFEF - G4
Encontrados 12 archivos parquet en Tercera RFEF - G4
Leído: teams_league_1184.parquet - 56 filas
Leído: team_stats_league_1184.parquet - 246 filas
Leído: events_league_1184.parquet - 235420 filas
Leído: names_players_league_1184.parquet - 2362 filas
Leído: players_league_1184.parquet - 2362 filas
Leído: matches_league_1184.parquet - 218 filas
Leído: sequence_data_league_1184.parquet - 68374 filas
Leído: leagues.parquet - 6 filas
Leído: player_stats_extended_league_1184.par

Guardado: ../../data_backup/archivos_parquet/sequence_data_league_all.parquet
Total filas en archivo combinado: 263349
Columnas: ['team_id', 'match_id', 'season_id', 'event_period', 'start_time', 'end_time', 'event_ids']

Combinando archivos con prefijo: teams_league
Guardado: ../../data_backup/archivos_parquet/teams_league_all.parquet
Total filas en archivo combinado: 263
Columnas: ['team_id', 'name', 'name_en', 'iso_country_code', 'player_ids', 'season_id']

Combinando archivos con prefijo: names_players_league
Guardado: ../../data_backup/archivos_parquet/names_players_league_all.parquet
Total filas en archivo combinado: 9619
Columnas: ['player_id', 'root_player_id', 'player_name', 'player_last_name', 'player_name_en', 'player_last_name_en', 'back_number', 'player_role', 'main_position', 'team_id', 'birth_date', 'season_id']

Combinando archivos con prefijo: events_league


Guardado: ../../data_backup/archivos_parquet/events_league_all.parquet
Total filas en archivo combinado: 904372
Columnas: ['id', 'event_period', 'event_time', 'match_id', 'team_id', 'player_id', 'x', 'y', 'attack_direction', 'relative_event.id', 'relative_event.event_time', 'relative_event.player_id', 'relative_event.x', 'relative_event.y', 'xg', 'ball_position.x', 'ball_position.y', 'season_id', 'eventType_event_type', 'outcome_event_type', 'cross_event_type', 'keyPass_event_type', 'assist_event_type', 'subEventType_event_type', 'bodyPart_event_type']

Combinando archivos con prefijo: team_stats_league
Guardado: ../../data_backup/archivos_parquet/team_stats_league_all.parquet
Total filas en archivo combinado: 946
Columnas: ['team_id', 'stats.goal', 'stats.total_shot', 'stats.shot_on_target', 'stats.shot_off_target', 'stats.shot_blocked', 'stats.shot_in_PA', 'stats.shot_outside_of_PA', 'stats.free_kick', 'stats.corner_kick', 'stats.throw_in', 'stats.penalty_kick', 'stats.pass', 'stats.